In [4]:
import pandas as pd
import os

In [8]:
dir_path = os.getcwd()
folder_path = f"{dir_path}/search_result"
file_name_list = os.listdir(folder_path)
len(file_name_list), file_name_list[0]

(11118, '17928.csv')

In [13]:
from tqdm import tqdm

all_df = pd.DataFrame()
dfs = []
for file_name in tqdm(file_name_list):
    # each file contain 10000 rows
    file_path = os.path.join(folder_path, file_name)
    df = pd.read_csv(file_path)
    dfs.append(df)
import time

start_time = time.time()
all_df = pd.concat(dfs, ignore_index=True)
print(f"--- {time.time() - start_time} seconds ---")

100%|██████████| 11118/11118 [00:32<00:00, 337.31it/s]


--- 0.7905001640319824 seconds ---


In [60]:
all_df.shape

(122776084, 3)

In [61]:
all_df.head()

,input_store_id,pred_store_id,score
0,17928,17928,0.000000
1,17928,13634,10.323901
2,17928,8232,11.813396
3,17928,16451,11.857945
4,17928,14529,12.068649


In [16]:
# save feather
all_df.to_feather("search_result.feather")

In [83]:
def get_top_n(df, input_store_id, n=11):
    df = df[df["input_store_id"] == input_store_id]
    df = df.sort_values(by="score", ascending=True)
    return df.head(n)

In [49]:
df_17928 = get_top_n(all_df, 17928)
# drop score
df_17928 = df_17928.drop(columns=["score"])
# expand pred_store_id to columns
df_17928
dfs = [df_17928]

In [53]:
store_id_list = all_df["input_store_id"].unique().tolist()
print(len(store_id_list))

11118


In [94]:
# group by input_store_id then expand pred_store_id to columns for 10 smallest score
test_df = all_df[:]
result_df = test_df.groupby("input_store_id").apply(
    lambda x: x.sort_values(by="score", ascending=True).head(11)
)
# change index name
result_df = result_df.reset_index(drop=True)
# display(result_df)
# remove row that input_store_id == pred_store_id
result_df = result_df[result_df["input_store_id"] != result_df["pred_store_id"]]
# for each group add new column with name rank and value is 1 to 10
result_df["rank"] = result_df.groupby("input_store_id").cumcount() + 1
# pivot pred_store_id with reserve order
result_df = result_df.pivot(
    index="input_store_id", columns="rank", values="pred_store_id"
)
result_df = result_df.reset_index()


# result_df = result_df.reset_index()
result_df

rank,input_store_id,1,2,3,4,5,6,7,8,9,10,11
0,7,11717.0,2230.0,8003.0,7064.0,5117.0,18045.0,9286.0,3418.0,7925.0,14309.0,NaN
1,10,814.0,5709.0,16166.0,17777.0,6904.0,17771.0,6199.0,136.0,15831.0,8003.0,NaN
2,12,4341.0,3258.0,781.0,3353.0,4602.0,14722.0,15423.0,10033.0,2212.0,1358.0,NaN
3,13,256.0,16188.0,12021.0,18232.0,16139.0,16479.0,6717.0,2923.0,11818.0,6906.0,NaN
4,14,14929.0,9834.0,14014.0,7438.0,9433.0,3425.0,102.0,8063.0,9187.0,5977.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
11113,18559,17894.0,16356.0,16119.0,17745.0,17361.0,7833.0,15669.0,10701.0,15421.0,18308.0,NaN
11114,18560,1619.0,6936.0,17832.0,15365.0,6882.0,14726.0,9423.0,4563.0,8359.0,18059.0,NaN
11115,18564,7596.0,2215.0,9451.0,11821.0,9424.0,14311.0,13110.0,9413.0,8446.0,7249.0,NaN
11116,18580,5486.0,16663.0,16615.0,4642.0,8997.0,14811.0,10019.0,1769.0,17460.0,3003.0,NaN


In [95]:
# drop column 11
result_df = result_df.drop(columns=[11])
# convert to int
result_df = result_df.astype(int)
# save result_df to csv
result_df.to_csv("result_top_10.csv", index=False)

In [106]:
selected_store_id_list_str = """1892
1892
1892
1892
1892
1892
1893
1893
1893
1893
1893
1893
1895
1895
1895
1895
1895
1895
1900
1900
1900
1900
1900
1900
1909
1909
1909
1909
1909
1909
1911
1911
1911
1911
1911
1911
1912
1912
1912
1912
1912
1912
1953
1953
1953
1953
1953
1953
1993
1993
1993
1993
1993
1993
2033
2033
2033
2033
2033
2033
2036
2036
2036
2036
2036
2036
2056
2056
2056
2056
2056
2056
2069
2069
2069
2069
2069
2069
2070
2070
2070
2070
2070
2070
2092
2092
2092
2092
2092
2092
2108
2108
2108
2108
2108
2108
2136
2136
2136
2136
2136
2136
2279
2279
2279
2279
2279
2279
2281
2281
2281
2281
2281
2281
2322
2322
2322
2322
2322
2322
2337
2337
2337
2337
2337
2337
2402
2402
2402
2402
2402
2402
2495
2495
2495
2495
2495
2495
2515
2515
2515
2515
2515
2515
2904
2904
2904
2904
2904
2904
16135
16135
16135
16135
16135
16135
2957
2957
2957
2957
2957
2957
3016
3016
3016
3016
3016
3016
3151
3151
3151
3151
3151
3151
3217
3217
3217
3217
3217
3217
3228
3228
3228
3228
3228
3228
3429
3429
3429
3429
3429
3429
3490
3490
3490
3490
3490
3490
3537
3537
3537
3537
3537
3537
3555
3555
3555
3555
3555
3555
3574
3574
3574
3574
3574
3574
3731
3731
3731
3731
3731
3731
3848
3848
3848
3848
3848
3848
3861
3861
3861
3861
3875
3875
3875
3875
3875
3875
3923
3923
3923
3923
3923
3923
3949
3949
3949
3949
3949
3949
3984
3984
3984
3984
3984
3984
4139
4139
4139
4139
4139
4139
4172
4172
4172
4172
4172
4172
4319
4319
4319
4319
4319
4319
4446
4446
4446
4446
4446
4446
4527
4527
4527
4527
4527
4527
4531
4531
4531
4533
4533
4533
4533
4533
4533
4596
4596
4596
4596
4596
4596
4648
4648
4648
4648
4648
4648
4923
4923
4923
4923
4923
4923
4990
4990
4990
4990
4990
4990
5145
5154
5154
5154
5154
5154
5154
5156
5156
5156
5156
5156
5156
5319
5319
5319
5319
5319
5319
5360
5360
5360
5360
5360
5360
5371
5371
5371
5371
5371
5371
5446
5446
5446
5446
5446
5446
5547
5547
5547
5547
5547
5547
5560
5560
5560
5560
5560
5560
5568
5568
5568
5568
5568
5568
5573
5573
5573
5573
5573
5573
5987
5987
5987
5987
5987
5987
5995
5995
5995
5995
5995
5995
6112
6112
6112
6112
6112
6112
6208
6208
6208
6208
6208
6208
6353
6353
6353
6353
6353
6353
6649
6649
6649
6649
6649
6649
6728
6728
6728
6728
6728
6728
6796
6796
6796
6796
6796
6796
6870
6870
6870
6870
6870
6870
6940
6940
6940
6998
6998
6998
6998
6998
6998
7149
7149
7149
7149
7149
7149
7214
7214
7214
7214
7214
7214
7270
7270
7270
7270
7270
7270
7342
7342
7342
7342
7342
7342
7344
7344
7344
7480
7480
7480
7480
7480
7480
7608
7608
7608
7608
7608
7608
7945
7945
7945
7945
7945
7945
7974
7974
7974
7974
7974
7974
8009
8009
8009
8009
8009
8009
8025
8025
8025
8025
8025
8025
8065
8065
8065
8065
8065
8065
8087
8087
8087
8087
8087
8087
8102
8102
8102
8102
8102
8102
8135
8135
8135
8135
8135
8421
8421
8421
8421
8421
8421
8578
8578
8578
8578
8578
8578
8602
8602
8602
8602
8602
8602
8668
8668
8668
8668
8668
8668
8703
8703
8703
8703
8703
8703
8707
8707
8707
8707
8707
8707
8780
8780
8780
8780
8780
8780
8836
8836
8836
8836
8836
8877
8877
8904
8904
8904
8904
8904
8904
8910
8910
8910
8910
8910
8910
8998
8998
8998
8998
8998
8998
9000
9000
9000
9000
9000
9000
9085
9085
9085
9085
9187
9187
9187
9187
9187
9187
9351
9351
9351
9351
9351
9351
9538
9538
9538
9538
9538
9538
9646
9646
9747
9747
9747
9747
9747
9747
9748
9748
9748
9748
9748
9748
9897
9897
9897
9897
9897
9897
10067
10067
10067
10067
10067
10067
10119
10119
10119
10119
10119
10119
10215
10215
10215
10215
10215
10215
10271
10271
10271
10271
10271
10271
10272
10272
10272
10272
10272
10272
10273
10273
10273
10273
10273
10273
10274
10274
10274
10274
10274
10274
10309
10309
10309
10309
10309
10309
10337
10337
10337
10337
10337
10337
10375
10375
10375
10375
10375
10375
10439
10439
10439
10439
10439
10439
10461
10461
10461
10461
10461
10461
10477
10477
10477
10477
10477
10477
10519
10519
10519
10519
10519
10519
10550
10550
10550
10550
10550
10550
10573
10573
10573
10573
10573
10573
10594
10594
10594
10594
10594
10594
10617
10617
10617
10617
10617
10617
10633
10633
10633
10633
10633
10633
11210
11210
11210
11210
11210
11210
10739
10739
10739
10739
10739
10739
10740
10740
10740
10740
10740
10740
10744
10744
10744
10744
10744
10744
10764
10764
10764
10764
10764
10764
10840
10840
10840
10840
10840
10840
10914
10914
10914
10914
10914
10914
10991
10991
10991
10991
10991
10991
11020
11020
11020
11020
11020
11020
11052
11052
11052
11052
11052
11052
11061
11061
11061
11061
11061
11061
11099
11099
11099
11099
11099
11099
11106
11106
11106
11106
11106
11106
11115
11115
11115
11115
11115
11115
11172
11172
11172
11172
11172
11172
11324
11324
11324
11324
11324
11324
11343
11343
11343
11343
11343
11343
11371
11371
11371
11371
11371
11371
11480
11480
11480
11480
11480
11480
11506
11506
11506
11506
11506
11506
11530
11530
11530
11530
11530
11530
11531
11531
11531
11531
11531
11531
11640
11640
11640
11640
11640
11640
11688
11688
11688
11688
11688
11688
11694
11694
11694
11694
11694
11694
11707
11707
11707
11707
11707
11707
11714
11714
11714
11714
11714
11714
11743
11743
11743
11743
11743
11743
11748
11748
11748
11748
11748
11748
11786
11786
11786
11786
11786
11786
11793
11793
11793
11793
11793
11793
11813
11813
11813
11813
11813
11813
11823
11823
11823
11823
11823
11823
12054
12054
12054
12054
12054
12054
13010
13010
13010
13010
13010
13010
13011
13011
13011
13011
13011
13011
13012
13012
13012
13012
13012
13012
13159
13159
13159
13159
13159
13159
13194
13194
13194
13194
13194
13194
13201
13201
13201
13201
13201
13201
13209
13209
13209
13209
13209
13209
13218
13218
13218
13218
13218
13218
13246
13246
13246
13246
13246
13246
13258
13258
13258
13258
13258
13258
13273
13273
13273
13273
13273
13273
13290
13290
13290
13290
13290
13290
13294
13294
13294
13294
13294
13294
13320
13320
13320
13320
13320
13320
13351
13351
13351
13351
13351
13351
13421
13421
13421
13421
13421
13421
13424
13424
13424
13424
13424
13424
13425
13425
13425
13425
13425
13425
13464
13464
13464
13464
13464
13464
13473
13473
13473
13473
13473
13473
13486
13486
13486
13486
13486
13486
13501
13501
13501
13501
13501
13501
13509
13509
13509
13509
13509
13509
13511
13511
13511
13511
13511
13511
13533
13533
13533
13533
13533
13533
13558
13558
13558
13558
13558
13558
13595
13595
13595
13595
13595
13595
13649
13649
13649
13649
13649
13649
13744
13744
13744
13744
13744
13744
13779
13779
13779
13779
13779
13779
13797
13797
13797
13797
13797
13797
13866
13866
13866
13866
13866
13866
13931
13931
13931
13931
13931
13931
13940
13940
13940
13940
13940
13940
13984
13984
13984
13984
13984
13984
13986
13986
13986
13986
13986
13986
14047
14047
14047
14047
14047
14047
14054
14054
14054
14054
14054
14054
14068
14068
14068
14068
14068
14068
14100
14100
14100
14100
14100
14100
14135
14135
14135
14135
14135
14135
14238
14238
14238
14238
14238
14238
14345
14345
14345
14345
14345
14345
14388
14388
14388
14388
14388
14388
14400
14400
14400
14400
14400
14400
14414
14414
14414
14414
14414
14414
14454
14454
14454
14454
14454
14454
14478
14478
14478
14478
14478
14478
14500
14500
14500
14500
14500
14500
14502
14502
14502
14502
14502
14502
14503
14503
14503
14503
14503
14503
14534
14534
14534
14534
14534
14534
14598
14598
14598
14598
14598
14598
14613
14613
14613
14613
14613
14613
14614
14614
14614
14614
14614
14614
14713
14713
14713
14713
14713
14713
14794
14794
14794
14794
14794
14794
14801
14801
14801
14801
14801
14801
14815
14815
14815
14815
14815
14815
14836
14836
14836
14836
14836
14836
14887
14887
14887
14887
14887
14887
14914
14914
14914
14914
14914
14914
14922
14922
14922
14922
14922
14922
14945
14945
14945
14945
14945
14945
14947
14947
14947
14947
14947
14947
14967
14967
14967
14967
14967
14967
15043
15043
15043
15043
15043
15043
15080
15080
15080
15080
15080
15080
15132
15132
15132
15132
15132
15132
15133
15133
15133
15133
15133
15133
15146
15146
15146
15146
15146
15146
15164
15164
15164
15164
15164
15164
15170
15170
15170
15170
15170
15170
15178
15178
15178
15178
15178
15178
15221
15221
15221
15221
15221
15221
15226
15226
15226
15226
15226
15226
15257
15257
15257
15257
15257
15257
15282
15282
15282
15282
15282
15282
15438
15438
15438
15438
15438
15438
15514
15514
15514
15514
15514
15514
15515
15515
15515
15515
15515
15515
15538
15538
15538
15538
15538
15538
15541
15541
15541
15541
15541
15541
15555
15555
15555
15555
15555
15555
15576
15576
15576
15576
15576
15576
15588
15588
15588
15588
15588
15588
15628
15628
15628
15628
15628
15628
15630
15630
15630
15630
15630
15630
15664
15664
15664
15664
15664
15664
15667
15667
15667
15667
15667
15667
15670
15670
15670
15670
15670
15670
15720
15720
15720
15720
15720
15720
15729
15729
15729
15729
15729
15729
15772
15772
15772
15772
15772
15772
15855
15855
15855
15855
15855
15855
15863
15863
15863
15863
15863
15863
15879
15879
15879
15879
15879
15879
15899
15899
15899
15899
15899
15899
15935
15935
15935
15935
15935
15935
15944
15944
15944
15944
15944
15944
15953
15953
15953
15953
15953
15953
15966
15966
15966
15966
15966
15966
16097
16097
16097
16097
16097
16097
16113
16113
16113
16113
16113
16113
16136
16136
16136
16136
16136
16136
16139
16139
16139
16139
16139
16139
16167
16167
16167
16167
16167
16167
16177
16177
16177
16177
16177
16177
16202
16202
16202
16202
16202
16202
16207
16207
16207
16207
16207
16207
16214
16214
16214
16214
16214
16214
16263
16263
16263
16263
16263
16263
16396
16396
16396
16396
16396
16396
16465
16465
16465
16465
16465
16465
16505
16505
16505
16505
16505
16505
16530
16530
16530
16530
16530
16530
16538
16538
16538
16538
16538
16538
16539
16539
16539
16539
16539
16539
16553
16553
16553
16553
16553
16553
16570
16570
16570
16570
16570
16570
16715
16715
16715
16787
16787
16788
16788
16788
438
438
438
438
438
438
555
555
555
555
555
555"""
selected_store_id_list = selected_store_id_list_str.split("\n")
selected_store_id_list = list(map(int, selected_store_id_list))
selected_store_id_list = list(set(selected_store_id_list))
selected_store_id_list[:10]

[8707, 15879, 2056, 13320, 11786, 14345, 10764, 16396, 11793, 9747]

In [107]:
# group by input_store_id then expand pred_store_id to columns for 10 smallest score
test_df = all_df[:]
test_df = test_df[test_df["input_store_id"].isin(selected_store_id_list)]
result_df = test_df.groupby("input_store_id").apply(
    lambda x: x.sort_values(by="score", ascending=True).head(11)
)
# change index name
result_df = result_df.reset_index(drop=True)
# display(result_df)
# remove row that input_store_id == pred_store_id
result_df = result_df[result_df["input_store_id"] != result_df["pred_store_id"]]
# for each group add new column with name rank and value is 1 to 10
result_df["rank"] = result_df.groupby("input_store_id").cumcount() + 1
# pivot pred_store_id with reserve order
result_df = result_df.pivot(
    index="input_store_id", columns="rank", values="pred_store_id"
)
result_df = result_df.reset_index()


# result_df = result_df.reset_index()
result_df

rank,input_store_id,1,2,3,4,5,6,7,8,9,10,11
0,438,17929.0,13227.0,10718.0,9403.0,6081.0,13638.0,15124.0,8248.0,13774.0,1802.0,NaN
1,555,3545.0,9720.0,5957.0,11302.0,1862.0,15023.0,8843.0,3543.0,8013.0,4637.0,NaN
2,1892,2658.0,1019.0,329.0,5269.0,14719.0,414.0,16417.0,921.0,4602.0,4358.0,NaN
3,1893,14249.0,10095.0,4995.0,1537.0,14107.0,14144.0,6175.0,11225.0,13501.0,16342.0,NaN
4,1895,7042.0,9080.0,18300.0,12088.0,3991.0,1756.0,15738.0,18167.0,14943.0,10392.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
285,16553,10864.0,7190.0,14545.0,6091.0,7133.0,4533.0,10255.0,2240.0,17255.0,2580.0,NaN
286,16570,503.0,902.0,18026.0,14328.0,15878.0,188.0,9852.0,16724.0,11323.0,11953.0,NaN
287,16715,441.0,16828.0,13225.0,17503.0,15698.0,981.0,1885.0,16451.0,8026.0,16270.0,NaN
288,16787,3063.0,8953.0,6252.0,2906.0,3002.0,17625.0,15162.0,11748.0,16530.0,687.0,NaN


In [108]:
test_df.shape

(3202470, 3)

In [109]:
# drop column 11
result_df = result_df.drop(columns=[11])
# convert to int
result_df = result_df.astype(int)
# save result_df to csv
result_df.to_csv("result_top_10_pttor.csv", index=False)